In [1]:
import sys
import os
sys.path.append("..")
import shap
import xgboost
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import colorsys
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn import preprocessing
from Framework.models import ShapleyModel
from Framework import cluster
from sklearn.cluster import KMeans
from Framework import datasets
from Framework import processing

In [2]:
notebook_mode = 'Original-PCA'
explainer_type = 'Linear'
model_type = 'XGBoost'
nClusters = 3
dataset_count = 9

In [3]:
X,y,name = datasets.returnDataset(dataset_count)

#X_train_pca,explained_var = processing.dimensional_reduce(PCA(n_components = 3),X_train)
#X_test_pca,_ = processing.dimensional_reduce(PCA(n_components = 3),X_test)

blackbox_model = ShapleyModel(explainer_type,model_type,nClusters,notebook_mode)

X_train_pca,X_train_tr_pca,X_train_val_pca,X_test_pca,y_train_pca,y_train_tr_pca,y_train_val_pca,y_test_pca = processing.prepare_pipeline_reduced_data(X,y,PCA(2))
X_train,X_train_tr,X_train_val,X_test,y_train,y_train_tr,y_train_val,y_test,_ = processing.prepare_pipeline_data(X,y)

scaler_y = preprocessing.StandardScaler().fit(y_train.reshape(-1,1))

y_train = scaler_y.transform(y_train.reshape(-1,1)).reshape(-1,)
y_test = scaler_y.transform(y_test.reshape(-1,1)).reshape(-1,)

shap_values = blackbox_model.explainShapley(X_train,y_train,X_train_tr,y_train_tr,X_train_val,y_train_val)
shap_dataframe = pd.DataFrame(data = shap_values,columns = X_train.columns)

#shap_dataframe_pca,explained_var_shap = processing.dimensional_reduce(PCA(n_components = 2),shap_dataframe)
#shap_dataframe_tsne,explained_var_tsne = processing.dimensional_reduce(TSNE(n_components=2),shap_dataframe)
shap_dataframe_pca = shap_dataframe

X_instanced = pd.concat((X_train,pd.DataFrame(columns = ['instance'])),axis = 1)
X_instanced['instance'].iloc[X_train_tr.index] = 'train'
X_instanced['instance'].iloc[X_train_val.index] = 'val'
y_instanced = pd.concat((pd.DataFrame(y_train,columns=['label']),pd.DataFrame(columns = ['instance'])),axis = 1)
y_instanced['instance'].iloc[X_train_tr.index] = 'train'
y_instanced['instance'].iloc[X_train_val.index] = 'val'

X_instanced_pca = pd.concat((X_train_pca,pd.DataFrame(columns = ['instance'])),axis = 1)
X_instanced_pca['instance'].iloc[X_train_tr.index] = 'train'
X_instanced_pca['instance'].iloc[X_train_val.index] = 'val'
y_instanced_pca = pd.concat((pd.DataFrame(y_train_pca,columns=['label']),pd.DataFrame(columns = ['instance'])),axis = 1)
y_instanced_pca['instance'].iloc[X_train_tr.index] = 'train'
y_instanced_pca['instance'].iloc[X_train_val.index] = 'val'



#Gather shapley values and output values in one dataframe

shap_instanced = pd.concat((pd.DataFrame(shap_dataframe_pca),pd.DataFrame(columns = ['instance'])),axis = 1)
shap_instanced['instance'].loc[X_train_tr.index] = 'train'
shap_instanced['instance'].loc[X_train_val.index] = 'val'


#Split the clusters into a dictionary
data_dict,data_dict_original,kmeans_shapley,kmeans_original = blackbox_model.clusterData(X_train_pca,shap_dataframe_pca)

y_train_df = pd.DataFrame(y_train)
_,_,original_split_shapley,y_shap = blackbox_model.prepareTrainData(data_dict,data_dict_original,X_instanced,y_instanced,shap_instanced,False)
original_split,y_org,_,_= blackbox_model.prepareTrainData(data_dict,data_dict_original,X_instanced_pca,y_instanced_pca,shap_instanced,False)

C:\Users\koral\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\koral\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\koral\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\koral\Anaconda3\lib\site-packages\sklearn\utils\validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\koral\Anaconda3\lib\site-packages\sklearn\pre

In [4]:
y_train[:5]

array([-0.10817444, -0.10817444, -2.3870493 , -0.10817444,  0.57548802])

In [5]:
original_split.head()

,0,1,instance,cluster
0,-0.767558,0.269491,train,0
1,0.345459,0.636127,train,0
2,-0.764202,0.777807,train,0
3,-0.236030,0.406245,train,0
4,-0.816334,-0.617274,train,2


In [6]:
org_shapley_reduced,_ = processing.reduce_after_clusters(PCA(2),original_split_shapley)
org_shapley_reduced.head()

,0,1,instance,cluster
0,-0.883632,0.763975,train,2
1,-0.282471,-1.204408,train,2
2,0.399835,-2.720378,train,2
3,-0.747790,-2.320361,train,2
4,-1.114077,1.387870,train,0


In [7]:
y_org.head()

,label,instance,cluster
0,-0.108174,train,0
1,-0.108174,train,0
2,-2.387049,train,0
3,-0.108174,train,0
4,0.575488,train,2


In [8]:
y_shap.head()

,label,instance,cluster
0,-0.108174,train,2
1,-0.108174,train,2
2,-2.387049,train,2
3,-0.108174,train,2
4,0.575488,train,0


In [9]:
model_dict,eval_results = blackbox_model.trainPredictor(original_split,y_org)
model_dict_shapley,eval_results_shapley = blackbox_model.trainPredictor(org_shapley_reduced,y_shap)

[0]	train-rmse:1.46153	val-rmse:1.47526
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.


C:\Users\koral\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\koral\Anaconda3\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Stopping. Best iteration:
[477]	train-rmse:0.300569	val-rmse:0.328328

[0]	train-rmse:0.826315	val-rmse:0.85336
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[506]	train-rmse:0.122016	val-rmse:0.250196

[0]	train-rmse:0.60046	val-rmse:0.629472
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[393]	train-rmse:0.138515	val-rmse:0.276412

[0]	train-rmse:0.671693	val-rmse:0.623992
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
Stopping. Best iteration:
[197]	train-rmse:0.303977	val-rmse:0.378298

[0]	train-rmse:2.0071	val-rmse:2.21924
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved 

In [10]:
shapley_test = blackbox_model.predictShapleyValues(X_test)
#shap_test_pca,_ = processing.dimensional_reduce(PCA(n_components = 2),shapley_test)
shapley_test_df = pd.DataFrame(shapley_test,columns = X_train.columns)
data_dict_shapley_test= blackbox_model.clusterDataTest(shap_dataframe_pca,kmeans_shapley.labels_,shapley_test_df)
data_dict_original_test= blackbox_model.clusterDataTest(X_train_pca,kmeans_original.labels_,X_test_pca)

In [11]:
data_dict_shapley_test.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,cluster
0,-0.017673,-0.022758,-0.017041,0.003709,-0.011321,-0.001849,0.050429,-0.064328,-0.001093,-0.007947,...,0.003024,0.002257,0.000213,0.021652,-0.010281,0.010286,-0.025739,0.097456,0.739328,0
1,-0.017673,-0.022758,-0.017041,-0.012346,-0.011321,-0.001849,0.010086,-0.064328,0.001322,0.002971,...,0.003024,0.002257,-0.001771,-0.020068,0.082541,0.034186,-0.087589,0.002796,0.071820,2
2,-0.017673,-0.022758,-0.074942,0.003709,-0.011321,-0.001849,0.050429,-0.019847,0.000115,-0.002488,...,0.003024,0.002257,-0.001771,0.021652,-0.041221,0.034186,0.051575,-0.060311,-0.150682,2
3,-0.017673,-0.022758,0.040861,0.003709,0.027565,-0.001849,0.050429,-0.019847,-0.001093,0.002971,...,0.059232,-0.006440,0.000213,0.021652,-0.010281,-0.061415,-0.056664,0.160562,0.961831,0
4,-0.017673,-0.022758,0.040861,0.003709,-0.011321,-0.001849,0.050429,0.024633,-0.000489,-0.002488,...,0.003024,0.010954,0.000213,0.021652,-0.041221,0.034186,-0.056664,0.129009,1.184334,0


In [12]:
y_test_df = pd.DataFrame(y_test_pca)
original_split_test,y_test_org,_,_ = blackbox_model.prepareTrainData(data_dict_shapley_test,data_dict_original_test,X_test_pca,y_test_df,shapley_test_df,True)
_,_,original_split_shapley_test,y_test_shap = blackbox_model.prepareTrainData(data_dict_shapley_test,data_dict_original_test,X_test,y_test_df,shapley_test_df,True)
original_split_shapley_test,_ = processing.reduce_after_clusters(PCA(2),original_split_shapley_test)

In [13]:
original_split_test.head()

,0,1,cluster
0,-0.250594,-0.953223,2
1,-0.818376,-0.008578,0
2,0.437642,0.414581,0
3,-0.537689,-1.411290,2
4,-0.532231,-1.386211,2


In [14]:
original_split_shapley_test.head()

,0,1,cluster
0,-2.243223,-0.291784,0
1,-0.672137,-0.233515,2
2,-1.655567,0.806709,2
3,-1.073807,-0.896001,0
4,-1.292733,-1.129312,0


In [15]:
preds_org = blackbox_model.predict(original_split_test,model_dict)
tot_rmse_org = blackbox_model.evaluate(preds_org,y_test_org)

In [16]:
preds_shap = blackbox_model.predict(original_split_shapley_test,model_dict_shapley)
tot_rmse_shap = blackbox_model.evaluate(preds_shap,y_test_shap)

In [17]:
y_train_tr = pd.DataFrame(y_train_tr_pca,index = X_train_tr_pca.index)
y_train_val = pd.DataFrame(y_train_val_pca,index = X_train_val_pca.index)
X_train_tr_pca.sort_index(inplace = True)
y_train_tr.sort_index(inplace = True)
X_train_val_pca.sort_index(inplace = True)
y_train_val.sort_index(inplace = True)

exp_model = blackbox_model.trainExtraModel(X_train_tr_pca,y_train_tr.iloc[:,0],X_train_val_pca,y_train_val.iloc[:,0])
preds_big = blackbox_model.predict(X_test_pca,exp_model)
tot_rmse_big = blackbox_model.evaluate(preds_big,y_test_pca)

C:\Users\koral\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\koral\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [18]:
names = ['Whole model','Original_ensemble','Shapley_ensemble']
values = [tot_rmse_big,tot_rmse_org[0],tot_rmse_shap[0]]
f=open("../Data/test_pca.txt", "a+")
f.write(name + ',')
[f.write('{0:.3f},'.format(values[v])) for v in range(len(values))]
f.write('{0},'.format(nClusters))
f.write('{0},'.format(notebook_mode))
f.write('{0},'.format(explainer_type))
f.write('{0}'.format(model_type))
f.write('\n')
f.close()